# Review DPS outputs

Make a mosaic of DPS outputs.

1. make a list of the DPS output paths with build_tindex.master.py
2. Identify duplicate tiles
3. Identify matching tiles; merge tindex.master with the original index tile file
4. make mosaicjson
6. View DPS results

In [ ]:
import geopandas
import pandas as pd
import os
import json
import collections

def local_to_s3(url, user = 'nathanmthomas', type='public'):
    ''' A Function to convert local paths to s3 urls'''
    if type is 'public':
        replacement_str = f's3://maap-ops-workspace/shared/{user}'
    else:
        replacement_str = f's3://maap-ops-workspace/{user}'
    return url.replace(f'/projects/my-{type}-bucket', replacement_str)

# Make a list of DPS outputs with build_tindex_master.py

In [ ]:
DPS_DATA_TYPE = 'AGB' #"Topo" "Landsat" "ATL08" "ATL08_filt" "AGB"
DPS_DATA_USER = 'lduncanson' #'nathanmthomas'
tindex_master_fn = f'/projects/shared-buckets/{DPS_DATA_USER}/DPS_tile_lists/{DPS_DATA_TYPE}_tindex_master.csv'
ATL08_filt_sample_tindex_master_fn = f'/projects/shared-buckets/{DPS_DATA_USER}/DPS_tile_lists/ATL08_filt_sample_tindex_master.csv'
Topo_tindex_master_fn = f'/projects/shared-buckets/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv'

Topo_tindex_master =   pd.read_csv(Topo_tindex_master_fn)
ATL08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv')
ATL08_filt_sample_tindex_master = pd.read_csv(ATL08_filt_sample_tindex_master_fn)

UPDATE_TINDEX = False

if not os.path.isfile(tindex_master_fn):
    UPDATE_TINDEX = True
else:
    print('Using existing tindex')
    print(tindex_master_fn)
    
if UPDATE_TINDEX:
    print(f"Building master tile index for: {DPS_DATA_TYPE}")
    dps_month = 10
    d_min = 14
    os.system(f"python /projects/icesat2_boreal/lib/build_tindex_master.py --type {DPS_DATA_TYPE} -m {dps_month} -d_min {d_min}")

In [ ]:
# Build up a dataframe from the list of dps output files
tindex_master = pd.read_csv(tindex_master_fn)
tindex_master['s3'] = [local_to_s3(local_path, user='lduncanson', type = 'private') for local_path in tindex_master['local_path']]
print(len(tindex_master))

# Get all covar tiles that should account for the set of output we want
tiles_covars = pd.read_csv(Topo_tindex_master_fn).tile_num
print(len(tiles_covars))

# Get all boreal tiles
boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg'
boreal_tile_index = geopandas.read_file(boreal_tile_index_path)
boreal_tile_index.astype({'layer':'int'})
boreal_tile_index.rename(columns={"layer":"tile_num"}, inplace=True)
boreal_tile_index["tile_num"] = boreal_tile_index["tile_num"].astype(int)

bad_tiles = [3540,3634,3728,3823,3916,4004] #Dropping the tiles near antimeridian that reproject poorly.

# For some reason, doing this causes 'MosaicJSON.from_features()' to fail...(below)
if True:
    # Remove bad tiles
    boreal_tile_index = boreal_tile_index[~boreal_tile_index['tile_num'].isin(bad_tiles)]
    
#print(boreal_tile_index.head())

# Identify duplicate tiles

In [ ]:
duplicate_tiles = [item for item, count in collections.Counter(tindex_master["tile_num"]).items() if count > 1]
print(duplicate_tiles)

# Identify completed, missing, failed, & duplicate tiles

In [ ]:
# The tiles we have:
# For the tindex_master, convert it into vector tiles that show the tiles we have
# Select the rows we have results for
tile_matches = boreal_tile_index.merge(tindex_master[~tindex_master['tile_num'].isin(bad_tiles)], how='right', on='tile_num')
#tile_matches_atl08_filt_samples = boreal_tile_index.merge(ATL08_filt_sample_tindex_master[~ATL08_filt_sample_tindex_master['tile_num'].isin(bad_tiles)], how='right', on='tile_num')
print(f'Completed: \t\t{len(tile_matches)}')

# Use the Topo tiles (COVAR set) to get the diff of what tiles we have and what we want
# MISSING TILES = DIFF(tiles_we_want, tiles_we_have)
tile_nums_missing = np.setdiff1d(tiles_covars, tile_matches.tile_num)

# The ATL08 tiles we have may not exactly match the set of COVAR tiles we have
# Use the set of ATL08 tiles we have, crossed with the tiles we're missing (based on COVAR set), to get tiles we're missing that we should definitely proces (b/c we have both COVAR and ATL08 tiles for them)
#tile_index_missing = boreal_tile_index.merge(boreal_tile_index[boreal_tile_index['tile_num'].isin(tile_nums_missing)], how='right', on='tile_num')
tile_index_missing = boreal_tile_index.merge(ATL08_filt_tindex_master[ATL08_filt_tindex_master['tile_num'].isin(tile_nums_missing)], how='right', on='tile_num')
print(f'Missing (w/ dup ATL08 filt): \t\t{len(tile_index_missing)}')
tile_index_missing.to_csv(f'/projects/my-public-bucket/DPS_tile_lists/Need_{DPS_DATA_TYPE}_tindex_master.csv')
#print(tile_index_missing.head())

# Meh, this doesnt give us fails
#tile_matches_failed = boreal_tile_index.merge(Topo_tindex_master[Topo_tindex_master['tile_num'].isin(tile_nums_missing)], how='right', on='tile_num')
#print(f'Missing b/c failed: \t{len(tile_matches_failed)}')

# Meh, duplicates are now removed in build_tindex_master
tile_matches_duplicates = boreal_tile_index.merge(Topo_tindex_master[Topo_tindex_master['tile_num'].isin(duplicate_tiles)], how='right', on='tile_num')
print(f'Duplicates: \t\t{len(tile_matches_duplicates)}')

# Drop duplicates
tile_matches = tile_matches.drop_duplicates(subset=['tile_num'], keep='last')

print(tile_matches.info())
tile_matches_geojson_string = tile_matches.to_crs("EPSG:4326").to_json()
tile_matches_geojson = json.loads(tile_matches_geojson_string)

tile_index_missing_geojson_string = tile_index_missing.to_crs("EPSG:4326").to_json()
tile_index_missing_geojson = json.loads(tile_index_missing_geojson_string)
tile_matches.head()

## Build a MosaicJSON

In [ ]:
from typing import Dict

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

def get_accessor(feature: Dict):
    """Return specific feature identifier."""
    return feature["properties"]["s3"]

In [ ]:
out_mosaic_json_fn = f's3://maap-ops-workspace/shared/{DPS_DATA_USER}/DPS_tile_lists/{DPS_DATA_TYPE}_tindex_master_mosaic.json' 

print(f"Building {out_mosaic_json_fn}")
mosaicdata = MosaicJSON.from_features(tile_matches_geojson.get('features'), minzoom=6, maxzoom=18, accessor=get_accessor)

with MosaicBackend(out_mosaic_json_fn, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)

## View the Results with Folium

In [ ]:
from folium import Map, TileLayer, GeoJson, LayerControl, Icon, Marker, features, Figure, CircleMarker

# Setup the mosaic tiling
tiler_base = "https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com/"
tiler_mosaic =  "".join([tiler_base, "mosaicjson/tiles/{z}/{x}/{y}"])

#### Set colormapping

In [ ]:
import branca.colormap as cm
import matplotlib.cm

max_AGB_display = 125

# TODO: find other valid 'colormap_names' for the tiler url that also work with cm.linear.xxxx.scale()
agb_colormap = 'viridis'#'RdYlGn_11' #'RdYlGn' #'nipy_spectral'
agb_tiles = f"{tiler_mosaic}?url=s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/AGB_tindex_master_mosaic.json&rescale=0,{max_AGB_display}&bidx=1&colormap_name={agb_colormap}"

agb_se_colormap = 'magma'
agb_se_tiles = f"{tiler_mosaic}?url=s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/AGB_tindex_master_mosaic.json&rescale=0,20&bidx=2&colormap_name={agb_se_colormap}"

colormap_AGB = cm.linear.viridis.scale(0, max_AGB_display).to_step(25)
colormap_AGB.caption = 'Mean of Aboveground Biomass Density [Mg/ha]'
colormap_AGB

#colormap_AGBSE = cm.linear.plasma.scale(0, 20).to_step(5)
#colormap_AGBSE.caption = 'Standard Error of Aboveground Biomass Density [Mg/ha]'

#colormap = cm.linear.nipy_spectral.scale(0, 125).to_step(25)
#colormap



In [ ]:
pal_height_cmap = cm.LinearColormap(colors = ['black','#636363','#fc8d59','#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850'], vmin=0, vmax=10)
pal_height_cmap.caption = 'Vegetation height from  ATL08 @ 30 m (h_can; rh98)'
pal_height_cmap

In [ ]:


# Get a basemap
tiler_basemap_gray = "http://services.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}"
tiler_basemap_image = 'https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'

# Get Vector layers
boreal_geojson = '/projects/shared-buckets/lduncanson/wwf_circumboreal_Dissolve.geojson'#'/projects/shared-buckets/nathanmthomas/boreal.geojson' 
boreal = geopandas.read_file(boreal_geojson)

if False:
    ecoboreal_geojson = '/projects/shared-buckets/nathanmthomas/Ecoregions2017_boreal_m.geojson'
    ecoboreal = geopandas.read_file(ecoboreal_geojson)
    # Reproject Vector Layers
    p1, p2, clat, clon = [50, 70, 40, 160]
    proj_str_aea = '+proj=aea +lat_1={:.2f} +lat_2={:.2f} +lat_0={:.2f} +lon_0={:.2f}'.format(p1, p2, clat, clon)
    ecoboreal_aea = ecoboreal.to_crs(proj_str_aea)
    # Apply a buffer
    ecoboreal_aea_buf = ecoboreal_aea["geometry"].buffer(1e5)
    # Go back to GCS
    ecoboreal_buf = ecoboreal_aea_buf.to_crs(boreal_tile_index.crs)
    #GeoJson(ecoboreal_aea_buf, name="Boreal extent from Ecoregions", style_function=lambda x:ecoboreal_style).add_to(m1)
    GeoJson(boreal, name="Boreal extent", style_function=lambda x:boreal_style).add_to(m1)

# Style Vector Layers
ecoboreal_style = {'fillColor': 'orange', 'color': 'orange'}
boreal_style = {'fillColor': 'gray', 'color': 'gray'}
boreal_subset_style = {'fillColor': 'red', 'color': 'red'}

# Map the Layers
Map_Figure=Figure(width=1500,height=800)
#------------------
m1 = Map(
    tiles="Stamen Toner",
    location=(60, 5),
    zoom_start=3
)
Map_Figure.add_child(m1)

boreal_tiles_style = {'fillColor': '#ff7f00', 'color': '#ff7f00'}
dps_subset_style = {'fillColor': '#377eb8', 'color': '#377eb8'}
dps_missing_style = {'fillColor': 'red', 'color': 'red'}

#GeoJson(atl08_gdf, name="ATL08"
#       ).add_to(m)

boreal_tile_index_layer = GeoJson(
        data=boreal_tile_index.to_crs("EPSG:4326").to_json(),
        style_function=lambda x:boreal_tiles_style,
        name="Boreal tiles",
        tooltip=features.GeoJsonTooltip(
            fields=['tile_num'],
            aliases=['Tile num:'],
        )
    )

tile_matches_layer = GeoJson(
        data=tile_matches_geojson,
        style_function=lambda x:dps_subset_style,
        name="AGB tiles: have",
        tooltip=features.GeoJsonTooltip(
            fields=['tile_num'],
            aliases=['Tile num:'],
        )
    )

if len(tile_index_missing) > 0:
    tile_matches_missing_layer = GeoJson(
            data=tile_index_missing_geojson,
            style_function=lambda x:dps_missing_style,
            name="AGB tiles: need"
        )

if True:
    basemaps = {
       'Google Terrain' : TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = False,
        control = True
       ),
        'basemap_gray' : TileLayer(
            tiles=tiler_basemap_gray,
            opacity=1,
            name="World gray basemap",
            attr="MAAP",
            overlay=False
        ),
        'Imagery' : TileLayer(
            tiles=tiler_basemap_image,
            opacity=1,
            name="Imagery",
            attr="MAAP",
            overlay=False
        ),
        'landsat_tiles_layer' : TileLayer(
            tiles= f"{tiler_mosaic}?url=s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Landsat_mosaic.json&rescale=0.01,0.5&bidx=6&colormap_name=viridis",
            opacity=1,
            name="landsat covars",
            attr="MAAP",
            overlay=False
        ),
        'topo_tiles_layer' : TileLayer(
            tiles= f"{tiler_mosaic}?url=s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_mosaic.json&rescale=0,1&bidx=3&colormap_name=bone",
            opacity=1,
            name="topo covars",
            attr="MAAP",
            overlay=False
        )
    }

agb_tiles_layer = TileLayer(
    tiles=agb_tiles,
    opacity=1,
    name="Boreal AGB",
    attr="MAAP",
    overlay=True
)

agb_se_tiles_layer = TileLayer(
    tiles=agb_se_tiles,
    opacity=1,
    name="Boreal AGB SE",
    attr="MAAP",
    overlay=True
)

# Add custom basemaps
basemaps['Google Terrain'].add_to(m1)
basemaps['basemap_gray'].add_to(m1)
basemaps['Imagery'].add_to(m1)

#basemaps['hs_gee'].add_to(m1)
if True:
    basemaps['landsat_tiles_layer'].add_to(m1)
    basemaps['topo_tiles_layer'].add_to(m1)

agb_tiles_layer.add_to(m1)
agb_se_tiles_layer.add_to(m1)

# Layers are added on top. Last layer is top layer
boreal_tile_index_layer.add_to(m1)
tile_matches_layer.add_to(m1)
tile_matches_missing_layer.add_to(m1)
#tile_matches_n_obs.add_to(m1)    
    
LayerControl().add_to(m1)

m1.add_child(colormap_AGB)
#m1.add_child(colormap_AGBSE)

m1

In [ ]:
def map_tile_n_obs(tindex_master_fn='s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv', 
                   map_name = '# of filtered ATL08 obs.',
                   max_n_obs=15000, map_width=1000, map_height=200, 
                   boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg'):
    
    import pandas as pd
    import geopandas
    import branca.colormap as cm
    from folium import Map, TileLayer, GeoJson, LayerControl, Icon, Marker, features, Figure, CircleMarker
    
    # Build up a dataframe from the list of dps output files
    tindex_master = pd.read_csv(tindex_master_fn)

    # Get all boreal tiles
    boreal_tile_index = geopandas.read_file(boreal_tile_index_path)
    #boreal_tile_index.astype({'layer':'int'})
    boreal_tile_index.rename(columns={"layer":"tile_num"}, inplace=True)
    boreal_tile_index["tile_num"] = boreal_tile_index["tile_num"].astype(int)

    bad_tiles = [3540,3634,3728,3823,3916,4004] #Dropping the tiles near antimeridian that reproject poorly.

    boreal_tile_index = boreal_tile_index[~boreal_tile_index['tile_num'].isin(bad_tiles)]
    tile_matches = boreal_tile_index.merge(tindex_master[~tindex_master['tile_num'].isin(bad_tiles)], how='right', on='tile_num')

    nobs_cmap = cm.LinearColormap(colors=cm.linear.RdYlGn_11.colors, vmin=0, vmax=max_n_obs)

    tile_matches['color'] = [nobs_cmap(n_obs) for n_obs in tile_matches.n_obs]

    Map_Figure3=Figure(width=map_width,height=map_height)
    
    m3 = Map(
        tiles="Stamen Toner",
        location=(60, 5),
        zoom_start=2
    )
    Map_Figure3.add_child(m3)

    tile_matches_n_obs = GeoJson(
        tile_matches,
        style_function=lambda feature: {
            'fillColor': feature['properties']['color'],
            #'color' : feature['properties']['color'],
            'color' : 'black',
            'weight' : 1,
            'fillOpacity' : 0.5,
            },
        name="ATL08 filt tiles: n_obs",
        tooltip=features.GeoJsonTooltip(
                fields=['tile_num','n_obs','local_path'],
                aliases=['Tile:','# obs.:','path:'],
            )
        )
    tile_matches_n_obs.add_to(m3)
    colormap_nobs= nobs_cmap.to_step(15)
    colormap_nobs.caption = map_name
    m3.add_child(colormap_nobs)

    LayerControl().add_to(m3)

    return m3

In [ ]:
map_tile_n_obs(tindex_master_fn='s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv', 
                   map_name = '# of filtered ATL08 obs. from night & day', max_n_obs=15000)

In [ ]:
map_tile_n_obs(tindex_master_fn='s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_sample_tindex_master.csv', 
                   map_name = '# of samples of filtered ATL08 obs. from night', max_n_obs=250)

In [ ]:
def map_tile(TILE_OF_INTEREST):
    DPS_DATA_TYPE = 'ATL08_filt' #"Topo" "Landsat" "ATL08" "AGB"
    DPS_DATA_USER = 'lduncanson' 
    #AGB_tindex_master_fn = f's3://maap-ops-workspace/shared/{DPS_DATA_USER}/DPS_tile_lists/AGB_tindex_master.csv'
    ATL08_filt_tindex_master_fn = f'/projects/shared-buckets/{DPS_DATA_USER}/DPS_tile_lists/ATL08_filt_tindex_master.csv'
    print(ATL08_filt_tindex_master_fn)

    # Build up a dataframe from the list of dps output files
    #AGB_tindex_master = pd.read_csv(AGB_tindex_master_fn)
    #AGB_tindex_master['s3'] = [local_to_s3(local_path, user=DPS_DATA_USER, type = 'private') for local_path in AGB_tindex_master['local_path']]

    ATL08_filt_tindex_master = pd.read_csv(ATL08_filt_tindex_master_fn)
    ATL08_filt_tindex_master['s3'] = [local_to_s3(local_path, user=DPS_DATA_USER, type = 'private') for local_path in ATL08_filt_tindex_master['local_path']]

    
    # Get the CSV fn for tile
    ATL08_filt_csv_fn = ATL08_filt_tindex_master['s3'].loc[ATL08_filt_tindex_master.tile_num == TILE_OF_INTEREST].tolist()[0]
    print(ATL08_filt_csv_fn)
    
    # Get corresponding ATL08 filtered csv
    atl08_df = pd.read_csv(ATL08_filt_csv_fn)
    atl08_gdf = geopandas.GeoDataFrame(atl08_df, crs="EPSG:4326", geometry = geopandas.points_from_xy(atl08_df.lon, atl08_df.lat) )
    print(f'\nNum. of ATL08 obs. in tile {TILE_OF_INTEREST}: \t{len(atl08_gdf)}')
    print(f'Percentage of water (ValidMask=0) ATL08 obs: \t\t{round(len(atl08_gdf[atl08_gdf.ValidMask == 0]) / len(atl08_gdf),4) *100}%')
    print(f'Percentage of water (slopemask=0) ATL08 obs: \t\t{round(len(atl08_gdf[atl08_gdf.slopemask == 0]) / len(atl08_gdf),4) *100}%')

    print(round(atl08_gdf.lat.mean(),4), round(atl08_gdf.lon.mean(),4))

    # Map the Layers
    Map_Figure=Figure(width=1000,height=600)
    #------------------
    m2 = Map(
        tiles="Stamen Terrain",
        location=(atl08_gdf.lat.mean(), atl08_gdf.lon.mean()),
        zoom_start=9
    )
    Map_Figure.add_child(m2)

    for lat, lon, ValidMask, slopemask, h_can in zip(atl08_gdf.lat, atl08_gdf.lon, atl08_gdf.ValidMask, atl08_gdf.slopemask, atl08_gdf.h_can):
        ATL08_obs = CircleMarker(location=[lat, lon],
                                radius = 10,
                                weight=0.25,
                                tooltip=str(round(h_can,2))+" m",
                                fill=True,
                                #fill_color=getfill(h_can),
                                color = pal_height_cmap(h_can),
                                #color = getcolor(ValidMask),
                                opacity=0
                                
                   )

        #Map_Figure.add_child(cm)
        ATL08_obs.add_to(m2)

    basemaps['Imagery'].add_to(m2)
    agb_tiles_layer.add_to(m2)

    # Layera are added underneath. Last layer is bottom layer
    #boreal_tile_index_layer.add_to(m2)
    #tile_matches_missing_layer.add_to(m2)
    #tile_matches_layer.add_to(m2)

    LayerControl().add_to(m2)
    m2.add_child(pal_height_cmap)
    
    return m2

In [ ]:
map_tile(4221)